# **SQL Server on Docker Runbook**

<img src="https://pngimage.net/wp-content/uploads/2018/06/sql-server-logo-png-3.png" width="300" height="100">
<img src="https://www.docker.com/sites/default/files/d8/2019-07/Moby-logo.png" width="150" height="100" > 

## Steps on Running SQL Server on Docker

### Pre-requisite
1. Download and Install Docker Desktop
https://hub.docker.com/editions/community/docker-ce-desktop-windows
https://docs.docker.com/docker-for-windows/install/
2. Create a login in Docker Hub
3. Enable Container and Hyper-V (Hyper-V Isolation) on Windows 10 Pro
https://docs.microsoft.com/en-us/virtualization/hyper-v-on-windows/quick-start/enable-hyper-v

### Running Docker
1. Download the image from the registry (Docker Hub) - *docker pull*
2. Create the container - *docker run*


### Check Docker Version

In [1]:
docker --version

Docker version 19.03.5, build 633a0ea


### Download SQL Server Images from MCR
Container Lifecycle
Build -> Push -> **PULL** -> Run

In [5]:
docker pull mcr.microsoft.com/mssql/server:2017-latest-ubuntu

2017-latest-ubuntu: Pulling from mssql/server
59ab41dd721a: Already exists
57da90bec92c: Already exists
06fe57530625: Already exists
5a6315cba1ff: Already exists
739f58768b3f: Already exists
3a58fde0fc61: Pulling fs layer
89b44069090d: Pulling fs layer
3f6b360deb9e: Pulling fs layer
b8d0242f03c2: Pulling fs layer
b8d0242f03c2: Waiting
3a58fde0fc61: Verifying Checksum
3a58fde0fc61: Download complete
3a58fde0fc61: Pull complete
b8d0242f03c2: Verifying Checksum
b8d0242f03c2: Download complete
89b44069090d: Verifying Checksum
89b44069090d: Download complete
89b44069090d: Pull complete
3f6b360deb9e: Verifying Checksum
3f6b360deb9e: Download complete
3f6b360deb9e: Pull complete
b8d0242f03c2: Pull complete
Digest: sha256:f551ff159479f402bc3a120d9bb509c78bbcb8d1fe401552e102f28bc654db0e
Status: Downloaded newer image for mcr.microsoft.com/mssql/server:2017-latest-ubuntu
mcr.microsoft.com/mssql/server:2017-latest-ubuntu


In [7]:
docker pull mcr.microsoft.com/mssql/server:2019-latest

2019-latest: Pulling from mssql/server
59ab41dd721a: Already exists
57da90bec92c: Already exists
06fe57530625: Already exists
5a6315cba1ff: Already exists
739f58768b3f: Already exists
3a58fde0fc61: Already exists
89b44069090d: Already exists
66c04823cbae: Pulling fs layer
e0d33b08f011: Pulling fs layer
e0d33b08f011: Verifying Checksum
e0d33b08f011: Download complete
66c04823cbae: Verifying Checksum
66c04823cbae: Download complete
66c04823cbae: Pull complete
e0d33b08f011: Pull complete
Digest: sha256:e6f8a9434cc64c6e2e96efd07771ff39664ee2e7444de04e942d60b4b73558e0
Status: Downloaded newer image for mcr.microsoft.com/mssql/server:2019-latest
mcr.microsoft.com/mssql/server:2019-latest


In [11]:
docker pull microsoft/mssql-server-windows-developer

Using default tag: latest
latest: Pulling from microsoft/mssql-server-windows-developer
3889bb8d808b: Pulling fs layer
449343c9d7e2: Pulling fs layer
08883151461d: Pulling fs layer
bafeb45a72fc: Pulling fs layer
f5c5aa235c5b: Pulling fs layer
158fead2ffa0: Pulling fs layer
746db9597cec: Pulling fs layer
9e96edbd8781: Pulling fs layer
c6dabab6234f: Pulling fs layer
975d0dccd859: Pulling fs layer
5b747cfb01b7: Pulling fs layer
c77992bbfd0f: Pulling fs layer
746db9597cec: Waiting
9e96edbd8781: Waiting
c6dabab6234f: Waiting
975d0dccd859: Waiting
5b747cfb01b7: Waiting
c77992bbfd0f: Waiting
bafeb45a72fc: Waiting
f5c5aa235c5b: Waiting
158fead2ffa0: Waiting
image operating system "windows" cannot be used on this platform


### List of All Downloaded Images
It only shows images related to container mode, i.e. if it is in linux container, it shows linux images EXCEPT running LCOW (Experimental Mode).

In [12]:
docker.exe images

REPOSITORY                       TAG                      IMAGE ID            CREATED             SIZE
mcr.microsoft.com/mssql/server   2019-latest              56655b462301        8 days ago          1.54GB
mcr.microsoft.com/mssql/server   2017-latest-ubuntu       a8343d3ce21c        3 weeks ago         1.39GB
mcr.microsoft.com/mssql/server   2019-GDR1-ubuntu-16.04   ba266fae5320        3 months ago        1.57GB
mcr.microsoft.com/mssql/server   2019-GA-ubuntu-16.04     76c7c66bff02        4 months ago        1.57GB


### **Create SQL Server 2017 in Linux Container**
-d Run container in background and print container ID  
-p mapping of external port to internal port  
-- name friendly customized name rather than system generated name  
-e environment variables; for SQL Server, two environment variables are needed.

In [13]:
docker run -d -p 1401:1433 --name sql1 -e "SA_PASSWORD=YourStrong@Passw0rd" -e "ACCEPT_EULA=Y" mcr.microsoft.com/mssql/server:2017-latest-ubuntu

ced94982e13469edd922d2db88d107010de6f20393daa85ec192095d6c0d712c


### **Create SQL Server 2019 Container in Linux Container**
For outside container, it cannot use ports already used. In this case, port 1401 is already used.

In [2]:
docker run -d -p 1402:1433 --name sql2 -e "SA_PASSWORD=YourStrong@Passw0rd" -e "ACCEPT_EULA=Y" mcr.microsoft.com/mssql/server:2019-latest

c556d0e186e8ed640b3ab5789243b373aa27750e5b4de671193eb9af1a0afaac


### How to Bring the Backup File to the Container?
Copy the backup file from the external host to the container

In [16]:
docker cp c:\tempoutlnx\AdventureWorksLT2017.bak sql2:/var/opt/mssql/data

Once the backup file is local to the container, we can run the restore using TSQL command (Switch Kernel to SQL and attach to SQL Server Instance)

In [1]:
restore database AdventureWorksLT2017 from disk = '/var/opt/mssql/data/AdventureWorksLT2017.bak' with move 'AdventureWorksLT2012_Data' to '/var/opt/mssql/data/adventureworkslt2017_data.mdf', move 'AdventureWorksLT2012_Log' to '/var/opt/mssql/data/adventureworkslt2017_log.ldf'

Processed 824 pages for database 'AdventureWorksLT2017', file 'AdventureWorksLT2012_Data' on file 1.

Processed 6 pages for database 'AdventureWorksLT2017', file 'AdventureWorksLT2012_Log' on file 1.

Converting database 'AdventureWorksLT2017' from version 869 to the current version 904.

Database 'AdventureWorksLT2017' running the upgrade step from version 869 to version 875.

Database 'AdventureWorksLT2017' running the upgrade step from version 875 to version 876.

...

Database 'AdventureWorksLT2017' running the upgrade step from version 901 to version 902.

Database 'AdventureWorksLT2017' running the upgrade step from version 902 to version 903.

Database 'AdventureWorksLT2017' running the upgrade step from version 903 to version 904.

RESTORE DATABASE successfully processed 830 pages in 0.230 seconds (28.192 MB/sec).

Total execution time: 00:00:05.820

### What Happens if Deleting the Container?
Will the Database still there?   
1. Let's delete the container. Before deleting it, the active container needs to stop.  
2. After delete the container, create the container using the same previous step.  
3. Check if the database is still there.

In [1]:
docker stop sql2
docker rm sql2

sql2
sql2


## **Persistent Storage Options**

1. Shared Volume
2. Mapped Volume

### Using Shared Volume

In [2]:
docker run -d -p 1403:1433 --name sql3 -v sqlvolume3:/var/opt/mssql -e "SA_PASSWORD=YourStrong@Passw0rd" -e "ACCEPT_EULA=Y" mcr.microsoft.com/mssql/server:2019-latest

7bf7daefdc06bb183914834f4463d7a833c6cda5ceb4589355bf2f02835f22e1


List all the volumes

In [4]:
docker volume ls

DRIVER              VOLUME NAME
local               sqlvolume3


Copy the backup file, run the restore to validate it is working.  
To check if the persistent stroage works:  
1. Delete the container.
2. Recreate the container and check if the database is still there.

In [5]:
docker cp c:\tempoutlnx\AdventureWorksLT2017.bak sql3:/var/opt/mssql/data

In [1]:
restore database AdventureWorksLT2017 from disk = '/var/opt/mssql/data/AdventureWorksLT2017.bak' with move 'AdventureWorksLT2012_Data' to '/var/opt/mssql/data/adventureworkslt2017_data.mdf', move 'AdventureWorksLT2012_Log' to '/var/opt/mssql/data/adventureworkslt2017_log.ldf'

Processed 824 pages for database 'AdventureWorksLT2017', file 'AdventureWorksLT2012_Data' on file 1.

Processed 6 pages for database 'AdventureWorksLT2017', file 'AdventureWorksLT2012_Log' on file 1.

Converting database 'AdventureWorksLT2017' from version 869 to the current version 904.

Database 'AdventureWorksLT2017' running the upgrade step from version 869 to version 875.

Database 'AdventureWorksLT2017' running the upgrade step from version 875 to version 876.

...

Database 'AdventureWorksLT2017' running the upgrade step from version 901 to version 902.

Database 'AdventureWorksLT2017' running the upgrade step from version 902 to version 903.

Database 'AdventureWorksLT2017' running the upgrade step from version 903 to version 904.

RESTORE DATABASE successfully processed 830 pages in 0.162 seconds (40.027 MB/sec).

Total execution time: 00:00:04.489

In [1]:
docker stop sql3
docker rm sql3

sql3
sql3


### Using Mapped Volume
Mapping the volume on outside to inside container

In [1]:
docker run -d -p 1404:1433 --name sql4 -v "c:\tempoutlnx:/tempinhost" -e "ACCEPT_EULA=Y" -e "MSSQL_SA_PASSWORD=YourStrong@Passw0rd" mcr.microsoft.com/mssql/server:2019-latest

: Msg 102, Level 15, State 1, Line 1
Incorrect syntax near '-'.

Total execution time: 00:00:00.013

Using TSQL to attach the database from the mapped volume

In [2]:
USE [master]
GO
CREATE DATABASE [AdventureWorks2017] ON 
( FILENAME = N'/tempinhost/AdventureWorks2017.mdf' ),
( FILENAME = N'/tempinhost/AdventureWorks2017_log.ldf' )
 FOR ATTACH
 GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:02.780

## **Running SQL Server on Windows Container**
Switch Docker to Windows Container.  
Create the container on the pre-download image.  
NOTE: SQL Server on Windows image has not been updated for a while still running SQL Server 2016.



In [1]:
docker run -e "ACCEPT_EULA=Y" -e "sa_password=YourStrong@Passw0rd" -p 1405:1433 --name sqlwin5  -v "c:\tempoutwin:c:\tempinhost" -d microsoft/mssql-server-windows-developer

809c7d36e3acdd4400698190c44d275833f9a376fbcf130a695f78a6f5e0347b


Attach the database files from outside storage using TSQL.

In [1]:
USE [master]
GO
CREATE DATABASE [AdventureWorks2017] ON 
( FILENAME = N'C:\tempinhost\AdventureWorks2017.mdf' ),
( FILENAME = N'C:\tempinhost\AdventureWorks2017_log.ldf' )
 FOR ATTACH
 GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.534

## **Clean Up**
1. Stop any running container.
2. Remove the container

In [2]:
docker stop sqlwin5
docker rm sqlwin5

sqlwin5
sqlwin5


In [3]:
docker stop sql1
docker rm sql1
docker stop sql2
docker rm sql2
docker stop sql3
docker rm sql3
docker stop sql4
docker rm sql4
docker volume rm sqlvolume3

sql1
sql1
sql2
sql2
sql3
sql3
sql4
sql4
sqlvolume3
